In [14]:
from tensorflow import keras

# mnist

def get_dataset(num_classes, rank=0, size=1):  
  (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data('MNIST-data-%d' % rank)
  x_train = x_train[rank::size]
  y_train = y_train[rank::size]
  x_test = x_test[rank::size]
  y_test = y_test[rank::size]
  x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
  x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255
  x_test /= 255
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes)
  return (x_train, y_train), (x_test, y_test)

In [17]:
from tensorflow.keras import models
from tensorflow.keras import layers

# t = models.Sequential()
# print(type(t))

def get_model(num_classes):
  model = models.Sequential()
  model.add(layers.Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=(28, 28, 1)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(num_classes, activation='softmax'))
  return model

In [18]:
# Specify training parameters
batch_size = 64
epochs = 3
num_classes = 10
 
def train(learning_rate=1.0):
  (x_train, y_train), (x_test, y_test) = get_dataset(num_classes)
  model = get_model(num_classes)
 
  # Specify the optimizer (Adadelta in this example), using the learning rate input parameter of the function so that Horovod can adjust the learning rate during training
  optimizer = keras.optimizers.Adadelta(lr=learning_rate)
 
  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
 
  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=2,
            validation_data=(x_test, y_test))
  return model

In [19]:
model = train(learning_rate=0.05)

C:\Users\haipw\Anaconda3\lib\site-packages\keras\optimizer_v2\adadelta.py:74: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adadelta, self).__init__(name, **kwargs)


Epoch 1/3
938/938 - 92s - loss: 0.6424 - accuracy: 0.8036 - val_loss: 0.2413 - val_accuracy: 0.9285 - 92s/epoch - 98ms/step
Epoch 2/3
938/938 - 89s - loss: 0.3334 - accuracy: 0.9013 - val_loss: 0.1847 - val_accuracy: 0.9459 - 89s/epoch - 95ms/step
Epoch 3/3
938/938 - 88s - loss: 0.2786 - accuracy: 0.9179 - val_loss: 0.1503 - val_accuracy: 0.9562 - 88s/epoch - 94ms/step


# setps
- model design
- strategy
  - chose loss
- algorithm
  - optimizer
- compile
- fit (train)
- evaluate (predict)

In [20]:
_, (x_test, y_test) = get_dataset(num_classes)
loss, accuracy = model.evaluate(x_test, y_test, batch_size=128)
print("loss:", loss)
print("accuracy:", accuracy)

79/79 [==============================] - 4s 46ms/step - loss: 0.1503 - accuracy: 0.9562
loss: 0.1502731740474701
accuracy: 0.9562000036239624


In [6]:
model.layers

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                        

In [21]:
model.save("cnn-mnist.h5")

In [ ]:
# onnx


In [13]:
import numpy as np

a = np.array([1,2,3,5])
print(type(a))
type(a[0])

<class 'numpy.ndarray'>


numpy.int32